<a href="https://colab.research.google.com/github/Dharmendra04/Blob-Detection/blob/main/Blob_Detection_AINT515.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download the file from Google Drive
file_id = '1jxvQ-C41deH6hFu9mvT1YfhGrNNzy12K'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('drop detection.zip')

# Extract the contents of the zip file
with zipfile.ZipFile('drop detection.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
import cv2 as cv   #opencv itself (under the name cv)
import numpy as np #numpy math library (under the name np)
import math
import time
#import cv2_imshow
#image displaying libraries
import pylab       
from matplotlib import pyplot as plt 
from IPython.display import HTML
from IPython.display import clear_output 
from base64 import b64encode
import pandas as pd

#make generated images appear inline below the code
%matplotlib inline 

#set the displayed image size
pylab.rcParams['figure.figsize'] = (10.0, 8.0) 

# Download example videos
!wget https://raw.githubusercontent.com/Jamesrogers221194/AINT-Files/3f989bb044250abdc3c25ccc5d4c10744e649369/AINT515/Coursework01/Video1%20for%20Vision%20CW.mp4
!wget https://raw.githubusercontent.com/Jamesrogers221194/AINT-Files/3f989bb044250abdc3c25ccc5d4c10744e649369/AINT515/Coursework01/Video2%20for%20Vision%20CW.mp4

--2023-05-11 12:09:22--  https://raw.githubusercontent.com/Jamesrogers221194/AINT-Files/3f989bb044250abdc3c25ccc5d4c10744e649369/AINT515/Coursework01/Video1%20for%20Vision%20CW.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 786074 (768K) [application/octet-stream]
Saving to: ‘Video1 for Vision CW.mp4’

Video1 for Vision C 100%[===================>] 767.65K  --.-KB/s    in 0.04s   

2023-05-11 12:09:22 (16.7 MB/s) - ‘Video1 for Vision CW.mp4’ saved [786074/786074]

--2023-05-11 12:09:22--  https://raw.githubusercontent.com/Jamesrogers221194/AINT-Files/3f989bb044250abdc3c25ccc5d4c10744e649369/AINT515/Coursework01/Video2%20for%20Vision%20CW.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.

## Blob Detector

In [ ]:
n=0 #blob counting (blobs counting)
k=0 #frame counting
m=0
#Load input video
videoInput = cv2.VideoCapture('/content/drop detection/Video1 for Vision CW.avi')
#videoInput = cv.VideoCapture('Video2 for Vision CW.mp4') #uncomment to load the second video
if (videoInput.isOpened() == False): 
  print("Error opening video stream or file")

#Setup output video writer
videoOutput = cv2.VideoWriter('output2.avi',cv2.VideoWriter_fourcc(*'MJPG'), 30, (int(videoInput.get(cv2.CAP_PROP_FRAME_WIDTH)),int(videoInput.get(cv2.CAP_PROP_FRAME_HEIGHT))))

#==================================Main Program Loop================================
while(True):
  #grab a frame, break the loop if there are no frames left
  ret, frame = videoInput.read()
  if(ret == False):
    break;
  else:
    k+=1

    if k>2:
      hsv_img = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

      #hsv_img = cv2.dilate(hsv_img, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4)))
      #hsv_img= cv2.erode(hsv_img, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4)))

      frame_filtered = cv2.inRange(hsv_img, np.array([10,0,0]), np.array([25,255,255])) #36#greyscale image

      #blob detector parameters (set to find squares)
      params = cv2.SimpleBlobDetector_Params()

      params.filterByArea = False
      params.minArea = 180

      params.filterByConvexity = True
      params.minConvexity = 0.60

      params.filterByInertia = False

      params.filterByCircularity = False
      params.minCircularity = 0.80
      params.maxCircularity = 1


      #create the detector and use it to find blobs in the thresholded image
      detector = cv2.SimpleBlobDetector_create(params)

      threshold_image = cv2.bitwise_not(frame_filtered)
      
      # Apply morphological erosion to the HSV image
      threshold_image = cv2.dilate(threshold_image, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4)))
      threshold_image= cv2.erode(threshold_image, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4)))
      
      #sorting the blobs according to the area size and getting the first 6
      blobs = detector.detect(threshold_image)
      blobs_sorted = sorted(blobs, key=lambda x: x.size, reverse=True)
      blobs_sort = blobs_sorted[:6]

      #draw detected blobs to the colour image
      input_image_RGB_blobs = cv2.drawKeypoints(frame, blobs_sort, np.array([]), (255,0,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

      
 #calculating number of blobs
      if len(blobs)>5:
       
       #if blob is detected within this area increase the blob_count by one
        blob_coords_lower = (400.0, 60.0) #lower coordinate
        blob_coords_upper =(550.0,100.0) #upper coordinate

        # Extract the first blob from the image
        for i in range (len(blobs_sort)):
          blob_coords1 = blobs_sort[i].pt[0], blobs_sort[i].pt[1], blobs_sort[i].size
          
          #to avoid smaller blobs
          if blobs[i].size >17:
            cv2.circle(input_image_RGB_blobs, (int(blob_coords1[0]), int(blob_coords1[1])), 3, (100, 0, 0), -1) #drawing the centre of blobs

          if blob_coords_upper[0] > blob_coords1[0] > blob_coords_lower[0] and blob_coords_upper[1] > blob_coords1[1] > blob_coords_lower[1]:
            n+=1
            print(f'number of sucessful blobs detected so far: {round(3+(n/(30*3)))}') #30 frames per seconds, 3 second approximate for each blob creation, and 3 initial blobs found in the start of the video

            
#predicitng the outer warp 
      rgb_image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #change the image colour space to HSV
      masked_image1 = cv2.inRange(input_image_RGB_blobs,(0,0,0),(50,50,50)) #setting the hsv colour range for black colour

      Grey_gaussian1 = cv2.GaussianBlur(masked_image1,(5,5),0) #applying gausian filter

      w= int(Grey_gaussian1.shape[1]/2) 
      Grey_gaussian_half = Grey_gaussian1[:,:w] #only considering the half of the frame to consider droplets only


      contours, hierarchy = cv2.findContours(Grey_gaussian_half, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) #ontaining the contours
      
      #sorting the contours and obtaining first 4
      sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)
      for i in range (4):
        contoured_img = cv2.drawContours(input_image_RGB_blobs, contours, -1, (0,0,255), 1) #-1 - all contours should be drawn
        M1 = cv2.moments(sorted_contours[i])

        # calculate x,y coordinate of center
        CX = int(M1["m10"] / (M1["m00"]))
        CY = int(M1["m01"] / (M1["m00"]))

        cv2.circle(input_image_RGB_blobs, (CX, CY), 5, (255, 255, 0), -1)
      
      # Write the processed frame to the output video file
      videoOutput.write(contoured_img)

    else:
      videoOutput.write(frame)
    

videoInput.release()  #turn off the video camera
cv2.destroyAllWindows()  #close the window showing the video frames 


number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of sucessful blobs detected so far: 3
number of 